In [1]:
%pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [87]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [88]:
import pyarrow as pa
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from source import data_import

In [89]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.preprocessing import StandardScaler

In [90]:
import tensorflow as tf
from tensorflow.keras.utils import plot_model

import keras 
from keras.models import Sequential # intitialize the ANN
from keras.layers import Dense, Activation, Dropout 

In [91]:
data = np.load('data/feature_extract_reshaped.npy')

In [92]:
data.shape

(30, 62, 77)

In [93]:
# Split data into train and test set

train_split_ratio = int((data.shape[2])*0.7)
#train_data, test_data = df[:train_test_split_ratio], df[train_test_split_ratio:]

In [94]:
train_data = data[:,:,:train_split_ratio]
test_data = data[:,:,train_split_ratio:]
test_data.shape

(30, 62, 24)

In [95]:
segments =train_data[:,1,:]
segments

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  1.,  1., ...,  1.,  1.,  1.],
       [ 2.,  2.,  2., ...,  2.,  2.,  2.],
       ...,
       [27., 27., 27., ..., 27., 27., 27.],
       [28., 28., 28., ..., 28., 28., 28.],
       [29., 29., 29., ..., 29., 29., 29.]])

In [96]:
target =train_data[:,0,:]
target

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]])

In [97]:
segments =train_data[:,1,:]
target =train_data[:,0,:]

In [98]:
target

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]])

In [99]:

X_train = train_data[:,1:,:]
y_train = train_data[:,0,:]
X_test = test_data[:,1:,:]
y_test = test_data[:,0,:]


In [100]:
X_train.shape

(30, 61, 53)

In [101]:
y_train.shape

(30, 53)

In [103]:
y_train[:,1]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [104]:
X_test.shape

(30, 61, 24)

In [105]:
y_test.shape

(30, 24)

In [118]:
X_train[:,:,2]

array([[0.00000000e+00, 4.94224146e-01, 1.27910249e-01, ...,
        1.03527436e-02, 5.91626409e-03, 1.86080175e-04],
       [1.00000000e+00, 5.64037570e-01, 1.26468872e-01, ...,
        7.74757871e-03, 5.79565343e-03, 2.41560424e-04],
       [2.00000000e+00, 5.81151603e-01, 3.37799273e-02, ...,
        2.83532072e-03, 6.72784337e-04, 1.70766497e-04],
       ...,
       [2.70000000e+01, 5.45042023e-01, 5.88232977e-01, ...,
        9.03008194e-02, 2.68103607e-02, 4.87157571e-04],
       [2.80000000e+01, 6.58733552e-01, 7.68754041e-01, ...,
        6.80688906e-02, 2.66971806e-02, 5.77551570e-04],
       [2.90000000e+01, 5.42060295e-01, 2.98420529e-01, ...,
        3.72417831e-02, 9.13425284e-03, 5.53634611e-04]])

In [117]:
y_train[:,2]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1.])

In [11]:
# Split into train and test set 
#X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state = 42)


In [111]:
X_train.shape[0]

30

In [114]:
# Define a sequential model
model = keras.Sequential()

# Add an input layer with an input shape of (time_steps, input_features)
model.add(layers.InputLayer(input_shape=(X_train.shape[0], 61)))

# Add a SimpleRNN layer with 32 units (neurons)
model.add(layers.SimpleRNN(32, activation='tanh'))

# Add the output layer with 1 neuron and sigmoid activation for binary classification
model.add(layers.Dense(1, activation='sigmoid'))

# Compile the model with binary cross-entropy loss for binary classification
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [86]:
X_train.shape

(30, 60, 53)

In [119]:
# Assuming X_train and y_train are your training data
model.fit(X_train[:,:,2], y_train[:,2], epochs=100, batch_size=30)

Epoch 1/100


ValueError: in user code:

    File "/Users/anabroggini/Documents/bootcamp/ai2/.venv/lib/python3.11/site-packages/keras/src/engine/training.py", line 1377, in train_function  *
        return step_function(self, iterator)
    File "/Users/anabroggini/Documents/bootcamp/ai2/.venv/lib/python3.11/site-packages/keras/src/engine/training.py", line 1360, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/anabroggini/Documents/bootcamp/ai2/.venv/lib/python3.11/site-packages/keras/src/engine/training.py", line 1349, in run_step  **
        outputs = model.train_step(data)
    File "/Users/anabroggini/Documents/bootcamp/ai2/.venv/lib/python3.11/site-packages/keras/src/engine/training.py", line 1126, in train_step
        y_pred = self(x, training=True)
    File "/Users/anabroggini/Documents/bootcamp/ai2/.venv/lib/python3.11/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filter